In [68]:
import pandas as pd
from sklearn.metrics import r2_score, mean_squared_log_error
import numpy as np

In [32]:
def loss_calc(position, combined=False):

    print("DISCLAIMER: Geno Smith excluded from Predictions")
    position = "QB"
    combined = False
    #load in fantasypros preseason rankings for one position
    position = position.upper()
    lower_position = position.lower()
    fan_proj = pd.read_csv("data/espn_and_nflcom_preseason_projs/FantasyPros_Fantasy_Football_Projections_"+position+".csv")

    #add in a column at beginning with preseason rank by using order of rows
    num_players = len(fan_proj)
    ranks = np.arange(1, num_players+1, 1)
    fan_proj.insert(0, "Experts_Rank", ranks)
    fan_proj.rename(columns={"NAME": "Player", "FPTS": "Experts_Projections"}, inplace=True)

    my_proj = None

    #load in projections from best model (check github for these)
    if combined == True:
        my_proj = pd.read_csv('draft_proj_083022/'+position+'/'+lower_position+'_combined_projs.csv')
    else:
        if position == "QB":
            my_proj= pd.read_csv('projections/QB/BayesianRidge2022_projections_20220830-142824.csv')
        elif position == "RB":
            my_proj= pd.read_csv('projections/RB/RandomForestRegressor2022_projections_20220830-142827.csv')
        elif position == "WR":
            my_proj= pd.read_csv('projections/WR/BayesianRidge2022_projections_20220830-142827.csv')
        else:
            my_proj= pd.read_csv('projections/TE/LinearRegression2022_projections_20220830-142828.csv')
    num_players = len(my_proj)
    ranks = np.arange(1, num_players+1, 1)
    my_proj.insert(0, "My_Rank", ranks)

    #load in fantasy actual results
    player_actuals = pd.read_csv("data/fantasy_actual_results/"+position+"_fantasy_actuals_2022.csv")
    num_players = len(player_actuals)
    ranks = np.arange(1, num_players+1, 1)
    player_actuals.insert(0, "Actual_Rank", ranks)
    player_actuals.rename(columns={"NAME": "Player", "FPTS": "Actual_FPTS"}, inplace=True)


    #create table to store loss statistics
    loss_stats = []

    #create df for desired columns
    results_df = pd.DataFrame()
    results_df["Player"] = player_actuals['Player']
    results_df["Actual_FPTS"] = player_actuals['Actual_FPTS']
    results_df["Actual_Rank"] = player_actuals['Actual_Rank']
    results_df = results_df.merge(fan_proj[['Player', 'Experts_Projections', 'Experts_Rank']], how='inner', on='Player')
    results_df = results_df.merge(my_proj[['Player', 'Model_Projection_Points', 'My_Rank']], how='inner', on='Player')
    results_df = results_df[['Player','Actual_FPTS','Experts_Projections','Model_Projection_Points','Actual_Rank','Experts_Rank','My_Rank']]


    #calculate loss metrics against Fantasy Football Rankings
    experts_points_r2 = r2_score(results_df['Experts_Projections'],results_df['Actual_FPTS'])
    print("Expert Points R2 Score: "+ str(experts_points_r2))
    my_points_r2 = r2_score(results_df['Model_Projection_Points'],results_df['Actual_FPTS'])
    print("My Points R2 Score: "+ str(my_points_r2))
    experts_ranks_r2 = r2_score(results_df['Experts_Rank'],results_df['Actual_Rank'])
    print("Expert Rank R2 Score: "+ str(experts_ranks_r2))
    my_ranks_r2 = r2_score(results_df['My_Rank'],results_df['Actual_Rank'])
    print("My Rank R2 Score: "+ str(my_ranks_r2))


    #store in stats table
    #calculate loss metrics against own model predictions
    #store in stats table
    #print stats table
    #return stats table
    #prediction_results = df
    #return prediction_results


In [33]:
loss_calc('qb')

Expert Points R2 Score: -4.002066004282719
My Points R2 Score: -0.8797829856818318
Expert Rank R2 Score: -0.4084411407333286
My Rank R2 Score: -0.6676394821795413


In [75]:
print("DISCLAIMER: Geno Smith excluded from Predictions")
position = "WR"
combined = False
#load in fantasypros preseason rankings for one position
position = position.upper()
lower_position = position.lower()
fan_proj = pd.read_csv("data/espn_and_nflcom_preseason_projs/FantasyPros_Fantasy_Football_Projections_"+position+".csv")

#add in a column at beginning with preseason rank by using order of rows
num_players = len(fan_proj)
ranks = np.arange(1, num_players+1, 1)
fan_proj.insert(0, "Experts_Rank", ranks)
fan_proj.rename(columns={"NAME": "Player", "FPTS": "Experts_Projections"}, inplace=True)

my_proj = None

#load in projections from best model (check github for these)
if combined == True:
    my_proj = pd.read_csv('draft_proj_083022/'+position+'/'+lower_position+'_combined_projs.csv')
else:
    if position == "QB":
        my_proj= pd.read_csv('projections/QB/BayesianRidge2022_projections_20220830-142824.csv')
    elif position == "RB":
        my_proj= pd.read_csv('projections/RB/RandomForestRegressor2022_projections_20220830-142827.csv')
    elif position == "WR":
        my_proj= pd.read_csv('projections/WR/BayesianRidge2022_projections_20220830-142827.csv')
    else:
        my_proj= pd.read_csv('projections/TE/LinearRegression2022_projections_20220830-142828.csv')
num_players = len(my_proj)
ranks = np.arange(1, num_players+1, 1)
my_proj.insert(0, "My_Rank", ranks)

#load in fantasy actual results
player_actuals = pd.read_csv("data/fantasy_actual_results/"+position+"_fantasy_actuals_2022.csv")
num_players = len(player_actuals)
ranks = np.arange(1, num_players+1, 1)
player_actuals.insert(0, "Actual_Rank", ranks)
player_actuals.rename(columns={"NAME": "Player", "FPTS": "Actual_FPTS"}, inplace=True)


#create table to store loss statistics
loss_stats = []

#create df for desired columns
results_df = pd.DataFrame()
results_df["Player"] = player_actuals['Player']
results_df["Actual_FPTS"] = player_actuals['Actual_FPTS']
results_df["Actual_Rank"] = player_actuals['Actual_Rank']
results_df = results_df.merge(fan_proj[['Player', 'Experts_Projections', 'Experts_Rank']], how='inner', on='Player')
results_df = results_df.merge(my_proj[['Player', 'Model_Projection_Points', 'My_Rank']], how='inner', on='Player')
results_df = results_df[['Player','Actual_FPTS','Experts_Projections','Model_Projection_Points','Actual_Rank','Experts_Rank','My_Rank']]


#calculate loss metrics against Fantasy Football Rankings
experts_points_rmsle = mean_squared_log_error(results_df['Experts_Projections'],results_df['Actual_FPTS'])
print("Expert Points RMSLE: "+ str(experts_points_rmsle))
my_points_rmsle = mean_squared_log_error(results_df['Model_Projection_Points'],results_df['Actual_FPTS'])
print("My Points RMSLE: "+ str(my_points_rmsle))
experts_ranks_rmsle = mean_squared_log_error(results_df['Experts_Rank'],results_df['Actual_Rank'])
print("Expert Rank RMSLE: "+ str(experts_ranks_rmsle))
my_ranks_rmsle = mean_squared_log_error(results_df['My_Rank'],results_df['Actual_Rank'])
print("My Rank RMSLE: "+ str(my_ranks_rmsle))


#store in stats table
#calculate loss metrics against own model predictions
#store in stats table
#print stats table
#return stats table
#prediction_results = df
#return prediction_results


DISCLAIMER: Geno Smith excluded from Predictions
Expert Points RMSLE: 0.17445679531028943
My Points RMSLE: 0.12382194147571238
Expert Rank RMSLE: 0.5858285325632647
My Rank RMSLE: 0.4062401077898617


In [76]:
results_df

,Player,Actual_FPTS,Experts_Projections,Model_Projection_Points,Actual_Rank,Experts_Rank,My_Rank
0,Justin Jefferson,368.66,194.6,286.705815,1,5,1
1,Tyreek Hill,342.90,165.4,234.207948,2,9,5
2,Davante Adams,335.50,191.5,255.579660,3,6,4
3,Stefon Diggs,316.60,161.0,220.693490,4,11,7
4,CeeDee Lamb,301.60,148.0,212.284122,5,17,10
5,A.J. Brown,299.60,172.4,199.393266,6,7,15
6,Amon-Ra St. Brown,267.60,145.5,224.383538,7,19,6
7,DeVonta Smith,254.60,126.1,179.546124,8,33,23
8,Jaylen Waddle,248.90,142.4,207.265525,9,23,12
9,Ja'Marr Chase,242.40,198.4,274.647401,10,4,2
